# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-4244339075-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Political Magazines explorative search") 

Consider the following exploratory scenario:


>  Investigate the U.S. Magazines which write about politics and their media presence



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P921`    | main subject  | predicate | 
| `wdt:P17`     | country       | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q41298`   | Magazine      | node |
| `wd:Q7163`    | politics      | node |
| `wd:Q30`      | U.S.          | node |
| `wd:Q217305`  | The New Yorker  | node |







Also consider

```
?p wdt:P17 wd:Q30 .
?p wdt:P31/wdt:P279* wd:Q41298  . 
```

is the BGP to retrieve all **type of publications in the U.S.**


The workload should


1. Identify the BGP for obtaining the US magazines that write about politics

2. Compare the number of social media followers and get the top five magazines along with main properties as the place of publication and subfields of work

3. Compare the number of followers with the top three US magazines writing mainly about sports

4. Get the name of notable employees working for The New Yorker and any other political magazine published in the US. Check if these employees have witten any book and if so get the title.  

5. Is any employee of The New Yorker ever been nominated for a prize or award?

In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT(?p)
WHERE { 

?p wdt:P17 wd:Q30 .
?p wdt:P31/wdt:P279* wd:Q41298  . 


} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '5422')]


1

## Part 1
### Identify the BGP for obtaining the US magazines that write about politics

### Retrieve data about Magazine

In [4]:
queryString = """
SELECT  ?p ?pName ?oName ?o
WHERE { 

    wd:Q41298 ?p ?o .
    OPTIONAL { ?p sc:name ?pName}
    OPTIONAL { ?o sc:name ?oName}

} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('pName', "topic's main Wikimedia portal"), ('oName', 'Portal:Periodicals'), ('o', 'http://www.wikidata.org/entity/Q15657033')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pName', 'described by source'), ('oName', 'Armenian Soviet Encyclopedia'), ('o', 'http://www.wikidata.org/entity/Q2657718')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('oName', 'Template:Infobox magazine'), ('o', 'http://www.wikidata.org/entity/Q5618602')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('pName', "topic's main template"), ('oName', 'Template:Infobox journal'), ('o', 'http://www.wikidata.org/entity/Q13256340')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('pName', 'properties for this type'), ('oName', 'Research Papers in Economics Series handle'), ('o', 'http://www.wikidata.org/entity/P2761')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('pName', 'subclass of'),

54

### Retrieve data with property o=magazine in US

In [5]:
queryString = """
SELECT ?s ?pName ?oName ?p ?o
WHERE { 

    ?s sc:name ?magazine ;
       wdt:P17 wd:Q30 ;
       ?p ?o .
    OPTIONAL {?p sc:name ?pName .}
    OPTIONAL {?o sc:name ?oName }
    FILTER regex(?magazine,".*[Mm]agazine.*")
} GROUP BY ?s
ORDER BY ?s ?pName ?oName ?p
LIMIT 100
  
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q100166024'), ('pName', 'archives at'), ('oName', 'University of Maryland Libraries'), ('p', 'http://www.wikidata.org/prop/direct/P485'), ('o', 'http://www.wikidata.org/entity/Q7895690')]
[('s', 'http://www.wikidata.org/entity/Q100166024'), ('pName', 'country'), ('oName', 'United States of America'), ('p', 'http://www.wikidata.org/prop/direct/P17'), ('o', 'http://www.wikidata.org/entity/Q30')]
[('s', 'http://www.wikidata.org/entity/Q100166024'), ('pName', 'dissolved, abolished or demolished date'), ('p', 'http://www.wikidata.org/prop/direct/P576'), ('o', '1978-01-01T00:00:00Z')]
[('s', 'http://www.wikidata.org/entity/Q100166024'), ('pName', 'headquarters location'), ('oName', 'Takoma Park'), ('p', 'http://www.wikidata.org/prop/direct/P159'), ('o', 'http://www.wikidata.org/entity/Q754271')]
[('s', 'http://www.wikidata.org/entity/Q100166024'), ('pName', 'inception'), ('p', 'http://www.wikidata.org/prop/direct/P571'), ('o', '1967-01-01T00:00:

100

### Search genre for magazine


In [6]:
queryString = """
SELECT DISTINCT ?genreName ?genre
WHERE { 
    ?s wdt:P17 wd:Q30 .
    ?s wdt:P31/wdt:P279* wd:Q41298  . 
    ?s wdt:P136 ?genre .
    OPTIONAL { ?genre sc:name ?genreName . }
    
} ORDER BY ASC(?genreName)
  
"""

print("Results")
run_query(queryString)

Results
[('genreName', 'Christian devotional literature'), ('genre', 'http://www.wikidata.org/entity/Q1348220')]
[('genreName', 'LGBTI+ related media'), ('genre', 'http://www.wikidata.org/entity/Q86463500')]
[('genreName', 'academic journal'), ('genre', 'http://www.wikidata.org/entity/Q737498')]
[('genreName', 'automobile magazine'), ('genre', 'http://www.wikidata.org/entity/Q3267592')]
[('genreName', 'big bust pornography'), ('genre', 'http://www.wikidata.org/entity/Q17379843')]
[('genreName', 'company magazine'), ('genre', 'http://www.wikidata.org/entity/Q12016659')]
[('genreName', 'computer magazine'), ('genre', 'http://www.wikidata.org/entity/Q3402519')]
[('genreName', 'cultural magazine'), ('genre', 'http://www.wikidata.org/entity/Q1791899')]
[('genreName', 'erotica'), ('genre', 'http://www.wikidata.org/entity/Q181001')]
[('genreName', 'experimental music'), ('genre', 'http://www.wikidata.org/entity/Q1640319')]
[('genreName', 'fashion magazine'), ('genre', 'http://www.wikidata.org

39

### Search for main subject wdt:P921

In [7]:
queryString = """
SELECT DISTINCT ?subjectName ?subject
WHERE { 
    ?s wdt:P17 wd:Q30 .
    ?s wdt:P31/wdt:P279* wd:Q41298  . 
    ?s wdt:P921 ?subject .
    OPTIONAL { ?subject sc:name ?subjectName . }
    
} ORDER BY ASC(?subjectName)
LIMIT 50
  
"""

print("Results")
run_query(queryString)

Results
[('subjectName', 'AEC industry'), ('subject', 'http://www.wikidata.org/entity/Q89358787')]
[('subjectName', 'African Americans'), ('subject', 'http://www.wikidata.org/entity/Q49085')]
[('subjectName', 'American Civil War'), ('subject', 'http://www.wikidata.org/entity/Q8676')]
[('subjectName', 'American studies'), ('subject', 'http://www.wikidata.org/entity/Q470125')]
[('subjectName', 'Araceae'), ('subject', 'http://www.wikidata.org/entity/Q48227')]
[('subjectName', 'Asian American'), ('subject', 'http://www.wikidata.org/entity/Q727928')]
[('subjectName', 'BDSM'), ('subject', 'http://www.wikidata.org/entity/Q190845')]
[('subjectName', 'Bethesda'), ('subject', 'http://www.wikidata.org/entity/Q584451')]
[('subjectName', 'Brigham Young University'), ('subject', 'http://www.wikidata.org/entity/Q332498')]
[('subjectName', 'California'), ('subject', 'http://www.wikidata.org/entity/Q99')]
[('subjectName', 'Category:Agricultural magazines'), ('subject', 'http://www.wikidata.org/entity/Q

50

### More specific search for subject

In [8]:
queryString = """
SELECT DISTINCT ?subjectName ?subject
WHERE { 
    ?s wdt:P17 wd:Q30 .
    ?s wdt:P31/wdt:P279* wd:Q41298  . 
    ?s wdt:P921 ?subject .
    OPTIONAL { ?subject sc:name ?subjectName . }
    FILTER regex(?subjectName,".*polit.*")
    
} ORDER BY ASC(?subjectName)

"""

print("Results")
run_query(queryString)

Results
[('subjectName', 'political science'), ('subject', 'http://www.wikidata.org/entity/Q36442')]
[('subjectName', 'politics'), ('subject', 'http://www.wikidata.org/entity/Q7163')]
[('subjectName', 'politics of the United States'), ('subject', 'http://www.wikidata.org/entity/Q330963')]


3

### More specific search for specific subject (try regex also with capital letter)

In [9]:
queryString = """
SELECT DISTINCT ?subjectName ?subject
WHERE { 
    ?s wdt:P17 wd:Q30 .
    ?s wdt:P31/wdt:P279* wd:Q41298  . 
    ?s wdt:P921 ?subject .
    OPTIONAL { ?subject sc:name ?subjectName . }
    FILTER regex(?subjectName,".*[Pp]olit.*")
    
} ORDER BY ASC(?subjectName)

"""

print("Results")
run_query(queryString)

Results
[('subjectName', 'political science'), ('subject', 'http://www.wikidata.org/entity/Q36442')]
[('subjectName', 'politics'), ('subject', 'http://www.wikidata.org/entity/Q7163')]
[('subjectName', 'politics of the United States'), ('subject', 'http://www.wikidata.org/entity/Q330963')]


3

### View title of US magazine about politic subjects

In [10]:
queryString = """
SELECT DISTINCT ?subjectName ?title
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P921 ?subject ;
       wdt:P1476 ?title .
    
    OPTIONAL { ?subject sc:name ?subjectName . }
    FILTER regex(?subjectName,".*polit.*")
    
} ORDER BY ASC(?subjectName)

"""

print("Results")
run_query(queryString)

Results
[('subjectName', 'political science'), ('title', 'Legislative Studies Quarterly')]
[('subjectName', 'political science'), ('title', 'Journal of World-Systems Research')]
[('subjectName', 'political science'), ('title', 'The Good Society')]
[('subjectName', 'political science'), ('title', 'Research & politics')]
[('subjectName', 'political science'), ('title', 'Bulletin of the atomic scientists')]
[('subjectName', 'political science'), ('title', 'Journal of Political Ecology')]
[('subjectName', 'political science'), ('title', 'Dissent')]
[('subjectName', 'political science'), ('title', 'Telos')]
[('subjectName', 'politics'), ('title', 'The Port Folio')]
[('subjectName', 'politics'), ('title', 'The New Yorker')]
[('subjectName', 'politics'), ('title', 'Politico magazine')]
[('subjectName', 'politics'), ('title', 'Commentary')]
[('subjectName', 'politics'), ('title', 'Jacobin')]
[('subjectName', 'politics'), ('title', 'The Liberator')]
[('subjectName', 'politics of the United Stat

15

### Result part 1
Found 15 US magazines about political subjects

## Part 2
### Compare the number of social media followers and get the top five magazines along with main properties as the place of publication and subfields of work

### Find top 5 magazines in U.S.
P8687 = social media followers 

In [11]:
queryString = """
SELECT DISTINCT ?title ?followers
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers .
       
} ORDER BY DESC(?followers)
 LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('title', 'Time'), ('followers', '17806837')]
[('title', 'Billboard'), ('followers', '10410725')]
[('title', 'Wired'), ('followers', '7900000')]
[('title', 'GQ'), ('followers', '6280000')]
[('title', 'Apparel Arts'), ('followers', '6280000')]


5

### Find top 5 magazines

In [12]:
queryString = """
SELECT DISTINCT ?title ?followers
WHERE { 
    ?s wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers .
       
} ORDER BY DESC(?followers)
 LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('title', 'The Economist'), ('followers', '25421540')]
[('title', 'Time'), ('followers', '17806837')]
[('title', 'Vice'), ('followers', '14500000')]
[('title', 'Billboard'), ('followers', '10410725')]
[('title', 'Vogue'), ('followers', '9770000')]


5

### Find top 5 magazines with their location
P159 headquarters location

P17 country 

P706 located in/on physical feature

In [13]:
queryString = """
SELECT DISTINCT ?title ?followers ?contry ?locatedIn ?location
WHERE { 
    ?s wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       wdt:P17 ?countryURI ;
       wdt:P159 ?located ;
       wdt:P706 ?in .
    OPTIONAL{ ?countryURI sc:name ?country . }
    OPTIONAL{ ?located sc:name ?locatedIn . }
    OPTIONAL{ ?in sc:name ?location . }
       
} ORDER BY DESC(?followers)
 LIMIT 5

"""

print("Results")
run_query(queryString)

Results
Empty


0

### Find top 5 magazines with their location (removing P706)
P159 headquarters location

P17 country

In [14]:
queryString = """
SELECT DISTINCT ?title ?followers ?country ?locatedIn
WHERE { 
    ?s wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       wdt:P17 ?countryURI ;
       wdt:P159 ?located .
    OPTIONAL{ ?countryURI sc:name ?country .}
    OPTIONAL{ ?located sc:name ?locatedIn . }
       
} ORDER BY DESC(?followers)
 LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('title', 'The Economist'), ('followers', '25421540'), ('country', 'United Kingdom'), ('locatedIn', 'London')]
[('title', 'Time'), ('followers', '17806837'), ('country', 'United States of America'), ('locatedIn', 'Manhattan')]
[('title', 'Billboard'), ('followers', '10410725'), ('country', 'United States of America'), ('locatedIn', 'New York City')]
[('title', 'Wired'), ('followers', '7900000'), ('country', 'United States of America'), ('locatedIn', 'San Francisco')]
[('title', 'GQ'), ('followers', '6280000'), ('country', 'United States of America'), ('locatedIn', 'New York City')]


5

### Find top 5 magazines with their location (adding optional)
P159 headquarters location

In [15]:
queryString = """
SELECT ?s ?title ?followers (GROUP_CONCAT(DISTINCT ?locatedIn; SEPARATOR = "," ) AS ?physicalLocations)
WHERE { 
    ?s wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       wdt:P159 ?located .
    OPTIONAL{ ?located <http://schema.org/name> ?locatedIn . }
    
}
GROUP BY ?s ?title ?followers
ORDER BY DESC(?followers)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q180089'), ('title', 'The Economist'), ('followers', '25421540'), ('physicalLocations', 'London')]
[('s', 'http://www.wikidata.org/entity/Q43297'), ('title', 'Time'), ('followers', '17806837'), ('physicalLocations', 'Manhattan')]
[('s', 'http://www.wikidata.org/entity/Q249838'), ('title', 'Vice'), ('followers', '14500000'), ('physicalLocations', 'New York City')]
[('s', 'http://www.wikidata.org/entity/Q485240'), ('title', 'Billboard'), ('followers', '10410725'), ('physicalLocations', 'New York City')]
[('s', 'http://www.wikidata.org/entity/Q154008'), ('title', 'Vogue'), ('followers', '9770000'), ('physicalLocations', 'Times Square')]


5

### Search more details for an entity (take the first one)

In [16]:
queryString = """
SELECT DISTINCT ?pName ?oName ?p ?o
WHERE { 
    wd:Q180089 ?p ?o
    OPTIONAL {?p sc:name ?pName}
    OPTIONAL {?o sc:name ?oName}
    
}

"""

print("Results")
run_query(queryString)

Results
[('pName', 'founded by'), ('oName', 'James Wilson'), ('p', 'http://www.wikidata.org/prop/direct/P112'), ('o', 'http://www.wikidata.org/entity/Q94765')]
[('pName', 'publisher'), ('oName', 'Economist Group'), ('p', 'http://www.wikidata.org/prop/direct/P123'), ('o', 'http://www.wikidata.org/entity/Q474153')]
[('pName', 'owned by'), ('oName', 'Economist Group'), ('p', 'http://www.wikidata.org/prop/direct/P127'), ('o', 'http://www.wikidata.org/entity/Q474153')]
[('pName', 'genre'), ('oName', 'print news magazine'), ('p', 'http://www.wikidata.org/prop/direct/P136'), ('o', 'http://www.wikidata.org/entity/Q1684600')]
[('pName', 'headquarters location'), ('oName', 'London'), ('p', 'http://www.wikidata.org/prop/direct/P159'), ('o', 'http://www.wikidata.org/entity/Q84')]
[('pName', 'country'), ('oName', 'United Kingdom'), ('p', 'http://www.wikidata.org/prop/direct/P17'), ('o', 'http://www.wikidata.org/entity/Q145')]
[('pName', 'place of publication'), ('oName', 'London'), ('p', 'http://ww

71

### Find top 5 magazines with their physical location and publication place
P159 headquarters location

P291 publication place

In [17]:
queryString = """
SELECT ?title ?followers (GROUP_CONCAT(DISTINCT ?locatedIn; SEPARATOR = "," ) AS ?physicalLocations) ?publicationPlace
WHERE { 
    ?s wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       wdt:P159 ?located ;
       wdt:P291 ?publicationPlaceURI .
    OPTIONAL{ ?located <http://schema.org/name> ?locatedIn . }
    OPTIONAL{ ?publicationPlaceURI sc:name ?publicationPlace }
    
}
GROUP BY ?s ?title ?followers ?publicationPlace
ORDER BY DESC(?followers)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('title', 'The Economist'), ('followers', '25421540'), ('physicalLocations', 'London'), ('publicationPlace', 'London')]
[('title', 'Time'), ('followers', '17806837'), ('physicalLocations', 'Manhattan'), ('publicationPlace', 'New York City')]
[('title', 'GQ'), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City')]
[('title', "Gentlemen's Quarterly"), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City')]
[('title', 'Apparel Arts'), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City')]


5

### Find top 5 magazines with their physical location and publication place
P159 headquarters location

P291 publication place

P276 location 

P101 field of works

P921 main subject

In [18]:
queryString = """
SELECT ?title ?followers ?location (GROUP_CONCAT(DISTINCT ?locatedIn; SEPARATOR = "," ) AS ?physicalLocations) ?publicationPlace
WHERE { 
    ?s wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       wdt:P159 ?located ;
       wdt:P276 ?locationURI ;
       wdt:P291 ?publicationPlaceURI .
    OPTIONAL{ ?located sc:name ?locatedIn . }
    OPTIONAL{ ?locationURI sc:name ?location . }
    OPTIONAL{ ?publicationPlaceURI sc:name ?publicationPlace }
    
}
GROUP BY ?s ?title ?followers ?publicationPlace ?location
ORDER BY DESC(?followers)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
Empty


0

### Find top 5 magazines with their physical location and publication place
P159 headquarters location

P291 publication place

P101 field of works

P921 main subject

In [19]:
queryString = """
SELECT ?title ?followers ?location (GROUP_CONCAT(DISTINCT ?locatedIn; SEPARATOR = "," ) AS ?physicalLocations) ?publicationPlace 
(GROUP_CONCAT(DISTINCT ?subject; SEPARATOR = "," ) AS ?mainSubjects) (GROUP_CONCAT(DISTINCT ?fieldName; SEPARATOR = "," ) AS ?fields)
WHERE { 
    ?s wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       wdt:P159 ?located ;
       wdt:P291 ?publicationPlaceURI ;
       wdt:P921 ?mainSubject ;
       wdt:P101 ?field .
    OPTIONAL{ ?located sc:name ?locatedIn . }
    OPTIONAL{ ?publicationPlaceURI sc:name ?publicationPlace }
    OPTIONAL{ ?mainSubject sc:name ?subject }
    OPTIONAL{ ?field sc:name ?fieldName }
    
}
GROUP BY ?s ?title ?followers ?publicationPlace ?location
ORDER BY DESC(?followers)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('title', 'Time'), ('followers', '17806837'), ('physicalLocations', 'Manhattan'), ('publicationPlace', 'New York City'), ('mainSubjects', 'news'), ('fields', 'news')]
[('title', 'GQ'), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City'), ('mainSubjects', "list of men's magazines"), ('fields', 'lifestyle')]
[('title', "Gentlemen's Quarterly"), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City'), ('mainSubjects', "list of men's magazines"), ('fields', 'lifestyle')]
[('title', 'Apparel Arts'), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City'), ('mainSubjects', "list of men's magazines"), ('fields', 'lifestyle')]
[('title', 'Scientific American'), ('followers', '3964337'), ('physicalLocations', 'Los Angeles'), ('publicationPlace', 'New York City'), ('mainSubjects', 'popular science'), ('fields', 'history')]


5

### Find top 5 magazines with their physical location and publication place
P101 field of works

In [20]:
queryString = """
SELECT ?title ?followers ?location (GROUP_CONCAT(DISTINCT ?locatedIn; SEPARATOR = "," ) AS ?physicalLocations) ?publicationPlace 
(GROUP_CONCAT(DISTINCT ?fieldName; SEPARATOR = "," ) AS ?fields)
WHERE { 
    ?s wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       wdt:P159 ?located ;
       wdt:P291 ?publicationPlaceURI ;
       wdt:P101 ?field .
    OPTIONAL{ ?located sc:name ?locatedIn . }
    OPTIONAL{ ?publicationPlaceURI sc:name ?publicationPlace }
    OPTIONAL{ ?field sc:name ?fieldName }
    
}
GROUP BY ?s ?title ?followers ?publicationPlace ?location
ORDER BY DESC(?followers)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('title', 'Time'), ('followers', '17806837'), ('physicalLocations', 'Manhattan'), ('publicationPlace', 'New York City'), ('fields', 'news')]
[('title', 'GQ'), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City'), ('fields', 'lifestyle')]
[('title', "Gentlemen's Quarterly"), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City'), ('fields', 'lifestyle')]
[('title', 'Apparel Arts'), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City'), ('fields', 'lifestyle')]
[('title', 'Scientific American'), ('followers', '3964337'), ('physicalLocations', 'Los Angeles'), ('publicationPlace', 'New York City'), ('fields', 'history')]


5

### Find top 5 magazines with their physical location and publication place
P921 main subject

In [21]:
queryString = """

SELECT ?title ?followers ?location (GROUP_CONCAT(DISTINCT ?locatedIn; SEPARATOR = "," ) AS ?physicalLocations) ?publicationPlace 
(GROUP_CONCAT(DISTINCT ?subject; SEPARATOR = "," ) AS ?mainSubjects)
WHERE { 
    ?s wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       wdt:P159 ?located ;
       wdt:P291 ?publicationPlaceURI ;
       wdt:P921 ?mainSubject .
    OPTIONAL{ ?located sc:name ?locatedIn . }
    OPTIONAL{ ?publicationPlaceURI sc:name ?publicationPlace }
    OPTIONAL{ ?mainSubject sc:name ?subject }
    
}
GROUP BY ?s ?title ?followers ?publicationPlace ?location
ORDER BY DESC(?followers)
LIMIT 5

"""

print("Results")
run_query(queryString)

Results
[('title', 'The Economist'), ('followers', '25421540'), ('physicalLocations', 'London'), ('publicationPlace', 'London'), ('mainSubjects', 'economics,politics')]
[('title', 'Time'), ('followers', '17806837'), ('physicalLocations', 'Manhattan'), ('publicationPlace', 'New York City'), ('mainSubjects', 'news')]
[('title', 'GQ'), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City'), ('mainSubjects', "list of men's magazines")]
[('title', "Gentlemen's Quarterly"), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City'), ('mainSubjects', "list of men's magazines")]
[('title', 'Apparel Arts'), ('followers', '6280000'), ('physicalLocations', 'New York City'), ('publicationPlace', 'New York City'), ('mainSubjects', "list of men's magazines")]


5

### Result part 2
In this section we can see that the top 5 magazines are changed in the different queries due to the fact that not all the information for the entities are always available

## Part 3
### Compare the number of followers with the top three US magazines writing mainly about sports

### Search US magazines and their topics

In [22]:
queryString = """
SELECT ?title ?followers (GROUP_CONCAT(DISTINCT ?argumentName; SEPARATOR=", ")  AS ?arguments)
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       (wdt:P136 | wdt:P921 ) ?argument .
    ?argument sc:name ?argumentName .
       
}
GROUP BY ?title ?followers
ORDER BY DESC(?followers)
LIMIT 20

"""

print("Results")
run_query(queryString)

Results
[('title', 'Time'), ('followers', '17806837'), ('arguments', 'news')]
[('title', 'Billboard'), ('followers', '10410725'), ('arguments', 'entertainment')]
[('title', 'Wired'), ('followers', '7900000'), ('arguments', 'business, computer magazine, cultural magazine, culture, entertainment, information technology, lifestyle, lifestyle magazine, popular science magazine')]
[('title', "Gentlemen's Quarterly"), ('followers', '6280000'), ('arguments', "list of men's magazines, men's magazine")]
[('title', 'Apparel Arts'), ('followers', '6280000'), ('arguments', "list of men's magazines, men's magazine")]
[('title', 'GQ'), ('followers', '6280000'), ('arguments', "list of men's magazines, men's magazine")]
[('title', 'Harvard Business Review'), ('followers', '5663315'), ('arguments', 'economics')]
[('title', 'Life'), ('followers', '4231609'), ('arguments', 'humour')]
[('title', 'Game informer magazine'), ('followers', '2334969'), ('arguments', 'video game, video game magazine')]
[('title

20

### Search US magazines and their topics
Filtering only sports magazinesP101

In [23]:
queryString = """
SELECT ?s ?title ?followers (GROUP_CONCAT(DISTINCT ?argumentName; SEPARATOR=", ")  AS ?arguments)
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       wdt:P8687 ?followers ;
       (wdt:P136 | wdt:P921 ) ?argument .
    ?argument sc:name ?argumentName .
    
    FILTER regex(?argumentName, ".*[Ss]port.*")
       
}
GROUP BY ?s ?title ?followers
ORDER BY DESC(?followers)


"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3108681'), ('title', 'Track & field news'), ('followers', '144716'), ('arguments', 'sports magazine')]
[('s', 'http://www.wikidata.org/entity/Q1140774'), ('title', 'The Ring'), ('followers', '104600'), ('arguments', 'sports magazine')]


2

### Search US magazines and their topics
Filtering only sports magazines

Try adding P101 field of work

In [24]:
queryString = """
SELECT ?title ?followers (GROUP_CONCAT(DISTINCT ?argumentName; SEPARATOR=", ")  AS ?arguments)
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       (wdt:P136 | wdt:P921 | wdt:P101 ) ?argument .
    ?argument sc:name ?argumentName .
    
    OPTIONAL{ ?s wdt:P8687 ?followers .}
    
    FILTER regex(?argumentName, ".*[Ss]port.*")
       
}
GROUP BY ?title ?followers
ORDER BY DESC(?followers)

"""

print("Results")
run_query(queryString)

Results
[('title', 'Track & field news'), ('followers', '144716'), ('arguments', 'sports magazine')]
[('title', 'The Ring'), ('followers', '104600'), ('arguments', 'sports magazine')]
[('title', 'Journal of Transport and Land Use'), ('arguments', 'transportation engineering')]
[('title', 'The orthopaedic journal of sports medicine'), ('arguments', 'sports medicine')]
[('title', 'Official Railway Guide'), ('arguments', 'rail transport in the United States')]
[('title', 'SPORT'), ('arguments', 'sports magazine')]
[('title', 'Orthopaedic journal of sports medicine'), ('arguments', 'sports medicine')]
[('title', 'Sport, Exercise, and Performance Psychology'), ('arguments', 'sport psychology')]
[('title', 'Soccer America'), ('arguments', 'sport')]
[('title', 'USA Today Sports Weekly'), ('arguments', 'sports magazine')]
[('title', 'The Official railway guide'), ('arguments', 'rail transport in the United States')]


11

### Adjust regex

In [25]:
queryString = """
SELECT ?title (GROUP_CONCAT(DISTINCT ?argumentName; SEPARATOR=", ")  AS ?arguments)
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       (wdt:P136 | wdt:P921 | wdt:P101 ) ?argument .
    ?argument sc:name ?argumentName .
    
    FILTER regex(?argumentName, "^[Ss]port.*")
       
}
GROUP BY ?title 

"""

print("Results")
run_query(queryString)

Results
[('title', 'The orthopaedic journal of sports medicine'), ('arguments', 'sports medicine')]
[('title', 'The Ring'), ('arguments', 'sports magazine')]
[('title', 'SPORT'), ('arguments', 'sports magazine')]
[('title', 'Orthopaedic journal of sports medicine'), ('arguments', 'sports medicine')]
[('title', 'Sport, Exercise, and Performance Psychology'), ('arguments', 'sport psychology')]
[('title', 'Soccer America'), ('arguments', 'sport')]
[('title', 'Track & field news'), ('arguments', 'sports magazine')]
[('title', 'USA Today Sports Weekly'), ('arguments', 'sports magazine')]


8

### Add followers with OPTIONAL and also title

In [26]:
queryString = """
SELECT ?title ?followers (GROUP_CONCAT(DISTINCT ?argumentName; SEPARATOR=", ")  AS ?arguments)
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       (wdt:P136 | wdt:P921 | wdt:P101) ?argument .
    OPTIONAL{?s wdt:P8687 ?followers .}
    ?argument sc:name ?argumentName .
    
    FILTER regex(?argumentName, "^[Ss]port*")
       
}
GROUP BY ?title ?followers
ORDER BY DESC(?followers)
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('title', 'Track & field news'), ('followers', '144716'), ('arguments', 'sports magazine')]
[('title', 'The Ring'), ('followers', '104600'), ('arguments', 'sports magazine')]
[('title', 'The orthopaedic journal of sports medicine'), ('arguments', 'sports medicine')]
[('title', 'SPORT'), ('arguments', 'sports magazine')]
[('title', 'Orthopaedic journal of sports medicine'), ('arguments', 'sports medicine')]


5

### Try to reach also other megazine without information  wdt:P136, wdt:P921, wdt:P101

In [27]:
queryString = """
SELECT ?title ?followers
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298 ;
       wdt:P1476 ?title . 
    OPTIONAL{?s wdt:P8687 ?followers .}
    
    FILTER NOT EXISTS { ?s (wdt:P136 | wdt:P921 | wdt:P101) ?argument . }
}
GROUP BY ?title ?followers
ORDER BY DESC(?followers)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('title', 'The Hollywood Reporter'), ('followers', '3280311')]
[('title', 'Inc.'), ('followers', '2799617')]
[('title', 'Inc'), ('followers', '2799617')]
[('title', 'The Atlantic Monthly'), ('followers', '2079799')]
[('title', 'The Atlantic'), ('followers', '2079799')]
[('title', 'New York'), ('followers', '1814011')]
[('title', 'Allure'), ('followers', '1680000')]
[('title', 'The Nation'), ('followers', '1279061')]
[('title', 'Golf digest'), ('followers', '897948')]
[('title', 'The intercept'), ('followers', '860009')]
[('title', 'Publishers Weekly'), ('followers', '804981')]
[('title', 'The publishers weekly'), ('followers', '804981')]
[('title', 'mental_floss'), ('followers', '775504')]
[('title', 'Mental floss'), ('followers', '775504')]
[('title', 'High Times'), ('followers', '723587')]
[('title', 'Psychology Today'), ('followers', '698339')]
[('title', 'Penthouse'), ('followers', '690392')]
[('title', 'The FADER'), ('followers', '669320')]
[('title', 'The FADER'), ('foll

20

### Add filter regex on titles

In [28]:
queryString = """
SELECT ?title ?followers
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298 ;
       wdt:P1476 ?title . 
    OPTIONAL{?s wdt:P8687 ?followers .}
    FILTER regex(?title,".*[sS]port.*")
    FILTER NOT EXISTS { ?s (wdt:P136 | wdt:P921 | wdt:P101) ?argument . }
}
GROUP BY ?title ?followers
ORDER BY DESC(?followers)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('title', 'Sports Illustrated'), ('followers', '357000')]
[('title', 'Index to the Sporting news'), ('followers', '225834')]
[('title', 'Sporting News'), ('followers', '225834')]
[('title', 'Sports illustrated for kids')]
[('title', 'Sports Illustrated Kids')]


5

### Search related entities without sport into the title but probably related

In [29]:
queryString = """
SELECT ?s ?title ?followers
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298 ;
       wdt:P1476 ?title . 
    OPTIONAL{?s wdt:P8687 ?followers .}

    FILTER NOT EXISTS { ?s (wdt:P136 | wdt:P921 | wdt:P101) ?argument . }
}
GROUP BY ?title ?followers
ORDER BY DESC(?followers)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q61503'), ('title', 'The Hollywood Reporter'), ('followers', '3280311')]
[('s', 'http://www.wikidata.org/entity/Q954455'), ('title', 'Inc'), ('followers', '2799617')]
[('s', 'http://www.wikidata.org/entity/Q954455'), ('title', 'Inc.'), ('followers', '2799617')]
[('s', 'http://www.wikidata.org/entity/Q1542536'), ('title', 'The Atlantic Monthly'), ('followers', '2079799')]
[('s', 'http://www.wikidata.org/entity/Q1542536'), ('title', 'The Atlantic'), ('followers', '2079799')]
[('s', 'http://www.wikidata.org/entity/Q1507987'), ('title', 'New York'), ('followers', '1814011')]
[('s', 'http://www.wikidata.org/entity/Q370099'), ('title', 'Allure'), ('followers', '1680000')]
[('s', 'http://www.wikidata.org/entity/Q1897630'), ('title', 'The Nation'), ('followers', '1279061')]
[('s', 'http://www.wikidata.org/entity/Q3110173'), ('title', 'Golf digest'), ('followers', '897948')]
[('s', 'http://www.wikidata.org/entity/Q15725297'), ('title', 'The interce

20

### Try to find a way to connect related magazines without wdt:P136, wdt:P921, wdt:P101 and without sport in the title

In [30]:
queryString = """
SELECT ?pName ?oName ?p ?o
WHERE { 
    wd:Q3110173 wdt:P17 wd:Q30 ;
                wdt:P31/wdt:P279* wd:Q41298 ;
                wdt:P1476 ?title ;
                ?p ?o .
    OPTIONAL{ ?p sc:name ?pName . }
    OPTIONAL{ ?o sc:name ?oName . }
}

"""

print("Results")
run_query(queryString)

Results
[('pName', 'country'), ('oName', 'United States of America'), ('p', 'http://www.wikidata.org/prop/direct/P17'), ('o', 'http://www.wikidata.org/entity/Q30')]
[('pName', 'different from'), ('oName', 'Golf digest'), ('p', 'http://www.wikidata.org/prop/direct/P1889'), ('o', 'http://www.wikidata.org/entity/Q56679051')]
[('pName', 'title'), ('p', 'http://www.wikidata.org/prop/direct/P1476'), ('o', 'Golf digest')]
[('pName', 'Twitter username'), ('p', 'http://www.wikidata.org/prop/direct/P2002'), ('o', 'golfdigest')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P236'), ('o', 'urn:ISSN:0017-176X')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('o', 'http://g.co/kg/m/07g8_w')]
[('pName', 'instance of'), ('oName', 'magazine'), ('p', 'http://www.wikidata.org/prop/direct/P31'), ('o', 'http://www.wikidata.org/entity/Q41298')]
[('pName', 'Instagram username'), ('p',

23

### Magazines with predicate P641 (sport) 

In [31]:
queryString = """
SELECT ?s ?title ?followers ?sportName
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298 ;
       wdt:P1476 ?title ; 
       wdt:P641 ?sport .
    OPTIONAL{?s wdt:P8687 ?followers .}
    ?sport sc:name ?sportName .
    

    FILTER NOT EXISTS { ?s (wdt:P136 | wdt:P921 | wdt:P101) ?argument . }
}
GROUP BY ?title ?followers
ORDER BY DESC(?followers)

"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3110173'), ('title', 'Golf digest'), ('followers', '897948'), ('sportName', 'golf')]
[('s', 'http://www.wikidata.org/entity/Q871928'), ('title', 'Baseball America'), ('followers', '278790'), ('sportName', 'baseball')]
[('s', 'http://www.wikidata.org/entity/Q951872'), ('title', 'Index to the Sporting news'), ('followers', '225834'), ('sportName', 'baseball')]
[('s', 'http://www.wikidata.org/entity/Q951872'), ('title', 'Sporting News'), ('followers', '225834'), ('sportName', 'baseball')]
[('s', 'http://www.wikidata.org/entity/Q2603450'), ('title', 'Swimming world and junior swimmer'), ('followers', '94158'), ('sportName', 'swimming')]
[('s', 'http://www.wikidata.org/entity/Q2603450'), ('title', 'Swimming World'), ('followers', '94158'), ('sportName', 'swimming')]
[('s', 'http://www.wikidata.org/entity/Q921844'), ('title', 'Slam'), ('sportName', 'basketball')]
[('s', 'http://www.wikidata.org/entity/Q2785076'), ('title', 'WWE Magazine'), ('sp

11

### Union of all the queries

In [32]:
queryString = """
SELECT ?s ?title ?followers
WHERE {
{
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298  ;
       wdt:P1476 ?title ;
       (wdt:P136 | wdt:P921 ) ?argument .
       ?argument sc:name ?argumentName . 
    OPTIONAL{ ?s wdt:P8687 ?followers .}
    
    FILTER regex(?argumentName, ".*[Ss]port.*")
}
UNION
{
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298 ;
       wdt:P1476 ?title . 
    OPTIONAL{?s wdt:P8687 ?followers .}
    FILTER regex(?title,".*[sS]port.*")
    FILTER NOT EXISTS { ?s (wdt:P136 | wdt:P921 | wdt:P101) ?argument . }
}
UNION
{
   ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298 ;
       wdt:P1476 ?title ; 
       wdt:P641 ?sport .
    OPTIONAL{?s wdt:P8687 ?followers .}
    FILTER NOT EXISTS { ?s (wdt:P136 | wdt:P921 | wdt:P101) ?argument . }
}
}
ORDER BY DESC(?followers)
LIMIT 3
"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q3110173'), ('title', 'Golf digest'), ('followers', '897948')]
[('s', 'http://www.wikidata.org/entity/Q275837'), ('title', 'Sports Illustrated'), ('followers', '357000')]
[('s', 'http://www.wikidata.org/entity/Q871928'), ('title', 'Baseball America'), ('followers', '278790')]


3

### Results part 3
It's strange that there is not any magazine about American Football or Basket in the top 3 because they are very followed sport in US... this resut is probabily biased because there are not magazines related to this field or they are not present in our dataset. 

## Part 4
### Get the name of notable employees working for The New Yorker and any other political magazine published in the US. Check if these employees have witten any book and if so get the title.

### Search The New Yorker URI

In [33]:
queryString = """
SELECT DISTINCT ?s ?subjectName ?title
WHERE { 
    ?s wdt:P17 wd:Q30 ;
       wdt:P31/wdt:P279* wd:Q41298 ;
       wdt:P921 ?subject ;
       wdt:P1476 ?title .
    
    OPTIONAL { ?subject sc:name ?subjectName . }
    FILTER regex(?subjectName,".*polit.*")
    
} ORDER BY ASC(?subjectName)

"""

print("Results")
run_query(queryString)

Results
[('s', 'http://www.wikidata.org/entity/Q6518264'), ('subjectName', 'political science'), ('title', 'Legislative Studies Quarterly')]
[('s', 'http://www.wikidata.org/entity/Q6296059'), ('subjectName', 'political science'), ('title', 'Journal of World-Systems Research')]
[('s', 'http://www.wikidata.org/entity/Q7737275'), ('subjectName', 'political science'), ('title', 'The Good Society')]
[('s', 'http://www.wikidata.org/entity/Q50815250'), ('subjectName', 'political science'), ('title', 'Research & politics')]
[('s', 'http://www.wikidata.org/entity/Q1004412'), ('subjectName', 'political science'), ('title', 'Bulletin of the atomic scientists')]
[('s', 'http://www.wikidata.org/entity/Q6295777'), ('subjectName', 'political science'), ('title', 'Journal of Political Ecology')]
[('s', 'http://www.wikidata.org/entity/Q2918282'), ('subjectName', 'political science'), ('title', 'Dissent')]
[('s', 'http://www.wikidata.org/entity/Q3512091'), ('subjectName', 'political science'), ('title',

15

### Find triples related to The New Yorker

In [34]:
queryString = """
SELECT DISTINCT ?pName ?oName ?p ?o
WHERE { 
    wd:Q217305 wdt:P17 wd:Q30 ;
               wdt:P31/wdt:P279* wd:Q41298  ;
               ?p ?o .
               
    OPTIONAL { ?p sc:name ?pName . }
    OPTIONAL { ?o sc:name ?oName . }
    
}

"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct-normalized/P236'), ('o', 'urn:ISSN:0028-792X')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('o', 'http://viaf.org/viaf/261263929')]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1042'), ('o', 'http://ld.zdb-services.de/resource/339293-4')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P236'), ('o', 'urn:ISSN:2163-3827')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P1015'), ('o', 'https://livedata.bibsys.no/authority/90374517')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P243'), ('o', 'http://www.worldcat.org/oclc/1760231')]
[('pName', 'Portuguese National Library ID'), ('p', 'http://www.wikidata.org/prop/direct/P1005'), ('o', '1695004')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P269'), ('o', 'http://www.idref.fr/118749498/id')]
[('p', 'http://www.wikidata.org/pr

86

### Find people

In [35]:
queryString = """
SELECT DISTINCT ?pName ?p ?o ?oName
WHERE { 
    ?p wdt:P31/wdt:P279* ?o .
    OPTIONAL{ ?o sc:name ?oName . }
    ?p sc:name ?pName .
    FILTER (regex(?o, "[Hh]uman") || regex(?oName,"[Hh]uman"))
    
}limit 10

"""

print("Results")
run_query(queryString)

Results
[('pName', 'Ciudad Vieja'), ('p', 'http://www.wikidata.org/entity/Q1007576'), ('o', 'http://www.wikidata.org/entity/Q486972'), ('oName', 'human settlement')]
[('pName', 'Fonsi Nieto'), ('p', 'http://www.wikidata.org/entity/Q1002728'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('pName', 'Lubomír Stoklásek'), ('p', 'http://www.wikidata.org/entity/Q100343342'), ('o', 'http://www.wikidata.org/entity/Q5'), ('oName', 'human')]
[('pName', 'Ciudad Vieja'), ('p', 'http://www.wikidata.org/entity/Q1007576'), ('o', 'http://www.wikidata.org/entity/Q16334295'), ('oName', 'group of humans')]
[('pName', 'Natuna'), ('p', 'http://www.wikidata.org/entity/Q10100'), ('o', 'http://www.wikidata.org/entity/Q811430'), ('oName', 'human-made geographic feature')]
[('pName', 'Natuna'), ('p', 'http://www.wikidata.org/entity/Q10100'), ('o', 'http://www.wikidata.org/entity/Q15642541'), ('oName', 'human-geographic territorial entity')]
[('pName', 'Natuna'), ('p', 'http://www.wikidata.or

10

### Try to find people related to The New Yorker

In [36]:
queryString = """
SELECT ?person ?personName (GROUP_CONCAT(DISTINCT ?pName; SEPARATOR=", ") AS ?works)
WHERE { 
    ?person ?p wd:Q217305 ;
            wdt:P31 wd:Q5 .
               
    OPTIONAL { ?person sc:name ?personName . }
    OPTIONAL { ?p sc:name ?pName . }
}
GROUP BY (?person) (?personName)
LIMIT 25
"""

print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q30104171'), ('personName', 'Keeanga-Yamahtta Taylor'), ('works', 'has written for')]
[('person', 'http://www.wikidata.org/entity/Q7614979'), ('personName', 'Steven Levitsky'), ('works', 'described by source')]
[('person', 'http://www.wikidata.org/entity/Q361617'), ('personName', 'E. B. White'), ('works', 'contributed to creative work')]
[('person', 'http://www.wikidata.org/entity/Q3719401'), ('personName', 'Ken Auletta'), ('works', 'has written for')]
[('person', 'http://www.wikidata.org/entity/Q5478671'), ('personName', 'Frances Gray Patton'), ('works', 'has written for')]
[('person', 'http://www.wikidata.org/entity/Q60025'), ('personName', 'Hannah Arendt'), ('works', 'employer')]
[('person', 'http://www.wikidata.org/entity/Q6306480'), ('personName', 'Julia Ioffe'), ('works', 'employer')]
[('person', 'http://www.wikidata.org/entity/Q105318410'), ('personName', 'Mansi Choksi'), ('works', 'has written for')]
[('person', 'http://www.wi

25

### Get URI for "has written for"

In [37]:
queryString = """
SELECT DISTINCT ?person ?personName ?pName ?p
WHERE { 
    ?person ?p wd:Q217305 ;
            wdt:P31 wd:Q5 .
               
    OPTIONAL { ?person sc:name ?personName . }
    OPTIONAL { ?p sc:name ?pName . }
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q16199458'), ('personName', 'Amanda Petrusich'), ('pName', 'has written for'), ('p', 'http://www.wikidata.org/prop/direct/P6872')]
[('person', 'http://www.wikidata.org/entity/Q22087892'), ('personName', 'Patrick Radden Keefe'), ('pName', 'has written for'), ('p', 'http://www.wikidata.org/prop/direct/P6872')]
[('person', 'http://www.wikidata.org/entity/Q548345'), ('personName', "John O'Hara"), ('pName', 'contributor to the creative work or subject'), ('p', 'http://www.wikidata.org/prop/direct/P767')]
[('person', 'http://www.wikidata.org/entity/Q124527'), ('personName', 'James Thurber'), ('pName', 'contributed to creative work'), ('p', 'http://www.wikidata.org/prop/direct/P3919')]
[('person', 'http://www.wikidata.org/entity/Q2550824'), ('personName', 'Mavis Gallant'), ('pName', 'contributed to creative work'), ('p', 'http://www.wikidata.org/prop/direct/P3919')]
[('person', 'http://www.wikidata.org/entity/Q60025'), ('personName', 'Hannah

10

### Search authors of The New Yorker 

In [38]:
queryString = """
SELECT DISTINCT ?person ?personName
WHERE { 
    ?person wdt:P6872 wd:Q217305 ;
            wdt:P31 wd:Q5 .
               
    OPTIONAL { ?person sc:name ?personName . }
} LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('person', 'http://www.wikidata.org/entity/Q16199458'), ('personName', 'Amanda Petrusich')]
[('person', 'http://www.wikidata.org/entity/Q22087892'), ('personName', 'Patrick Radden Keefe')]
[('person', 'http://www.wikidata.org/entity/Q520072'), ('personName', 'Andy Borowitz')]
[('person', 'http://www.wikidata.org/entity/Q273168'), ('personName', 'Susan Orlean')]
[('person', 'http://www.wikidata.org/entity/Q1077842'), ('personName', 'Chris Ware')]
[('person', 'http://www.wikidata.org/entity/Q2934539'), ('personName', 'Calvin Tomkins')]
[('person', 'http://www.wikidata.org/entity/Q5363086'), ('personName', 'Elizabeth Kolbert')]
[('person', 'http://www.wikidata.org/entity/Q7375285'), ('personName', 'Roz Chast')]
[('person', 'http://www.wikidata.org/entity/Q316607'), ('personName', 'Jonathan Franzen')]
[('person', 'http://www.wikidata.org/entity/Q4678940'), ('personName', 'Adam Davidson')]


10

### Find relations for the first one

In [39]:
queryString = """
SELECT DISTINCT ?p ?pName ?oName ?o
WHERE { 
    wd:Q16199458 ?p ?o .
    OPTIONAL { ?p sc:name ?pName . }
    OPTIONAL { ?o sc:name ?oName . }
} ORDER BY ASC(?pName)
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('o', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P214'), ('o', 'http://viaf.org/viaf/6846321')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('o', 'https://id.loc.gov/authorities/names/n2007060311')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('o', 'http://g.co/kg/m/0c3zs78')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P7704'), ('o', 'http://data.europeana.eu/agent/base/141770')]
[('p', 'http://schema.org/name'), ('o', 'Amanda Petrusich')]
[('p', 'http://www.wikidata.org/prop/direct/P7704'), ('pName', 'Europeana entity'), ('o', 'agent/base/141770')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pName', 'Freebase ID'), ('o', '/m/0c3zs78')]
[('p', 'http://www.wikidata.org/prop/direct/P6594'), ('pName', 'Guggenheim fellows ID'), ('o', 'amanda-petrusich')]
[('p', 'http://www.wikidata.org/prop/direct/P213'), ('pName', 'IS

33

### Find book instance

In [40]:
queryString = """
SELECT DISTINCT ?instance ?instanceName
WHERE { 

    ?s  wdt:P31 ?instance .
    ?instance sc:name ?instanceName .
    
    FILTER regex(?instanceName,".*[Bb]ook.*")
    
}LIMIT 15
"""

print("Results")
run_query(queryString)

Results
[('instance', 'http://www.wikidata.org/entity/Q60520'), ('instanceName', 'sketchbook')]
[('instance', 'http://www.wikidata.org/entity/Q22976748'), ('instanceName', 'stamp book')]
[('instance', 'http://www.wikidata.org/entity/Q26853769'), ('instanceName', 'workbook')]
[('instance', 'http://www.wikidata.org/entity/Q21818614'), ('instanceName', 'liturgical book of the Roman Rite')]
[('instance', 'http://www.wikidata.org/entity/Q283057'), ('instanceName', 'list of books of the Bible')]
[('instance', 'http://www.wikidata.org/entity/Q2477770'), ('instanceName', 'Ultrabook')]
[('instance', 'http://www.wikidata.org/entity/Q22759'), ('instanceName', 'book censorship')]
[('instance', 'http://www.wikidata.org/entity/Q538552'), ('instanceName', 'Book Fair')]
[('instance', 'http://www.wikidata.org/entity/Q106581242'), ('instanceName', 'booklet')]
[('instance', 'http://www.wikidata.org/entity/Q1599788'), ('instanceName', 'phrase book')]
[('instance', 'http://www.wikidata.org/entity/Q1027825'

15

### Try to connect book and The New Yorker's workers 

In [41]:
queryString = """
SELECT ?personName ?book
WHERE { 

    ?book  wdt:P31 ?instance .
    ?instance sc:name ?instanceName .
    FILTER regex(?instanceName,".*[Bb]ook.*")
    
    ?book ?writtenBy ?worker .
    
    ?worker wdt:P6872 wd:Q217305 ;
            wdt:P31 wd:Q5 .  
               
    OPTIONAL { ?worker sc:name ?personName . }
    
    
}
"""

print("Results")
run_query(queryString)

Results
[('personName', 'Patrick Radden Keefe'), ('book', 'http://www.wikidata.org/entity/Q106808347')]
[('personName', 'Chris Ware'), ('book', 'http://www.wikidata.org/entity/Q3179029')]
[('personName', 'Chris Ware'), ('book', 'http://www.wikidata.org/entity/Q2823374')]
[('personName', 'Chris Ware'), ('book', 'http://www.wikidata.org/entity/Q4986509')]
[('personName', 'Ed Yong'), ('book', 'http://www.wikidata.org/entity/Q28934718')]
[('personName', 'Alex Ross'), ('book', 'http://www.wikidata.org/entity/Q20875838')]
[('personName', 'Ronan Farrow'), ('book', 'http://www.wikidata.org/entity/Q71711397')]
[('personName', 'Louis Menand'), ('book', 'http://www.wikidata.org/entity/Q87147658')]
[('personName', 'Emily Witt'), ('book', 'http://www.wikidata.org/entity/Q50381841')]
[('personName', 'Kevin Young'), ('book', 'http://www.wikidata.org/entity/Q105609120')]
[('personName', 'Anna Wiener'), ('book', 'http://www.wikidata.org/entity/Q85811894')]


11

### See books' name 

In [42]:
queryString = """
SELECT DISTINCT ?personName ?bookTitle
WHERE { 

    ?book  wdt:P31 ?instance ;
           ?writtenBy ?worker .
    ?instance sc:name ?instanceName .
    FILTER regex(?instanceName,".*[Bb]ook.*")
    OPTIONAL{ ?book sc:name ?bookTitle }
    
    ?worker wdt:P6872 wd:Q217305 ;
            wdt:P31 wd:Q5 .  
               
    OPTIONAL { ?worker sc:name ?personName . }
    
    
}
"""

print("Results")
run_query(queryString)

Results
[('personName', 'Chris Ware'), ('bookTitle', 'Acme Novelty Library')]
[('personName', 'Chris Ware'), ('bookTitle', 'Jimmy Corrigan, the Smartest Kid on Earth')]
[('personName', 'Alex Ross'), ('bookTitle', 'Spin Alternative Record Guide')]
[('personName', 'Chris Ware'), ('bookTitle', 'Building Stories')]
[('personName', 'Emily Witt'), ('bookTitle', 'Properly Disposed')]
[('personName', 'Louis Menand'), ('bookTitle', 'Discovering Modernism : T. S. Eliot and his context')]
[('personName', 'Kevin Young'), ('bookTitle', 'Furious Flower : African American Poetry from the Black Arts Movement to the Present')]
[('personName', 'Ed Yong'), ('bookTitle', 'I Contain Multitudes')]
[('personName', 'Ronan Farrow'), ('bookTitle', 'Catch and Kill: Lies, Spies, and a Conspiracy to Protect Predators')]
[('personName', 'Anna Wiener'), ('bookTitle', 'Uncanny Valley')]
[('personName', 'Patrick Radden Keefe'), ('bookTitle', 'Empire of Pain')]


11

### See all books by authors who works for politc magazines

In [43]:
queryString = """
SELECT DISTINCT ?magazineName ?personName (GROUP_CONCAT(DISTINCT ?bookTitle; separator="; ") AS ?books)
WHERE { 

    ?book  wdt:P31 ?instance ;
           ?writtenBy ?worker .
    ?instance sc:name ?instanceName .
    FILTER regex(?instanceName,".*[Bb]ook.*")
    OPTIONAL{ ?book sc:name ?bookTitle }
    
    ?worker wdt:P6872 ?magazine ;
            wdt:P31 wd:Q5 ;
            sc:name ?personName . 
    
    ?magazine wdt:P17 wd:Q30 ;
              wdt:P31/wdt:P279* wd:Q41298  ;
              wdt:P921 ?subject .
    OPTIONAL{ ?magazine wdt:P1476 ?magazineName . }
    
    OPTIONAL { ?subject sc:name ?subjectName . }
    FILTER regex(?subjectName,".*polit.*")
    
} GROUP BY ?magazineName ?personName
"""

print("Results")
run_query(queryString)

Results
[('magazineName', 'The New Yorker'), ('personName', 'Ed Yong'), ('books', 'I Contain Multitudes')]
[('magazineName', 'The New Yorker'), ('personName', 'Chris Ware'), ('books', 'Acme Novelty Library; Building Stories; Jimmy Corrigan, the Smartest Kid on Earth')]
[('magazineName', 'The New Yorker'), ('personName', 'Anna Wiener'), ('books', 'Uncanny Valley')]
[('magazineName', 'The New Yorker'), ('personName', 'Patrick Radden Keefe'), ('books', 'Empire of Pain')]
[('magazineName', 'The New Yorker'), ('personName', 'Louis Menand'), ('books', 'Discovering Modernism : T. S. Eliot and his context')]
[('magazineName', 'The New Yorker'), ('personName', 'Emily Witt'), ('books', 'Properly Disposed')]


6

### Set not bound predicate for workers

In [44]:
queryString = """
SELECT DISTINCT ?magazineName ?personName (GROUP_CONCAT(DISTINCT ?bookTitle; separator="; ") AS ?books)
WHERE { 

    ?book  wdt:P31 ?instance ;
           ?writtenBy ?worker .
    ?instance sc:name ?instanceName .
    FILTER regex(?instanceName,".*[Bb]ook.*")
    OPTIONAL{ ?book sc:name ?bookTitle }
    
    ?worker ?works ?magazine ;
            wdt:P31 wd:Q5 ;
            sc:name ?personName . 
    
    ?magazine wdt:P17 wd:Q30 ;
              wdt:P31/wdt:P279* wd:Q41298  ;
              wdt:P921 ?subject .
    OPTIONAL{ ?magazine wdt:P1476 ?magazineName . }
    
    OPTIONAL { ?subject sc:name ?subjectName . }
    FILTER regex(?subjectName,".*polit.*")
    
} GROUP BY ?magazineName ?personName
"""

print("Results")
run_query(queryString)

Results
[('magazineName', 'The New Yorker'), ('personName', 'Lewis Mumford'), ('books', 'Herman Melville; Renewal of Life')]
[('magazineName', 'The New Yorker'), ('personName', 'Ed Yong'), ('books', 'I Contain Multitudes')]
[('magazineName', 'The New Yorker'), ('personName', 'Ronan Farrow'), ('books', 'Catch and Kill: Lies, Spies, and a Conspiracy to Protect Predators')]
[('magazineName', 'The New Yorker'), ('personName', 'Blake Eskin'), ('books', 'A life in pieces')]
[('magazineName', 'The New Yorker'), ('personName', 'Chris Ware'), ('books', 'Acme Novelty Library; Building Stories; Jimmy Corrigan, the Smartest Kid on Earth')]
[('magazineName', 'The New Yorker'), ('personName', 'Anna Wiener'), ('books', 'Uncanny Valley')]
[('magazineName', 'The New Yorker'), ('personName', 'Hannah Arendt'), ('books', 'Crises of the Republic; The Three Escapes of Hannah Arendt')]
[('magazineName', 'The New Yorker'), ('personName', 'Patrick Radden Keefe'), ('books', 'Empire of Pain')]
[('magazineName', 

15

### Up to here I assumed that the notable worker are who have written a book
It is strange that in the final result, without any restrictions on "The New Yorker", we see that the (political field) magazines worker are only belonging from "The New Yorker".

### New part after the Part 5 results
The following queries are done searching notable worker based on their nomination for a prize or award and then search if they wrote books.

In [45]:
queryString = """
SELECT ?magazineName ?workerName (GROUP_CONCAT(DISTINCT ?bookTitle; separator="; ") AS ?books)
WHERE { 
             
    ?worker ?works ?magazine ;
            wdt:P31 wd:Q5 ;
            wdt:P1411 ?price .
    OPTIONAL { ?worker sc:name ?workerName . }
            
    ?magazine wdt:P17 wd:Q30 ;
              wdt:P31/wdt:P279* wd:Q41298 ;
              wdt:P921 ?subject .
    OPTIONAL{ ?magazine wdt:P1476 ?magazineName . }
    ?subject sc:name ?subjectName .
    FILTER regex(?subjectName,".*polit.*")
    
    ?book  wdt:P31 ?instance ;
           ?writtenBy ?worker .
    ?instance sc:name ?instanceName .
    FILTER regex(?instanceName,".*[Bb]ook.*")
    OPTIONAL{ ?book sc:name ?bookTitle }
    
    
} GROUP BY ?magazineName ?workerName
"""

print("Results")
run_query(queryString)

Results
[('magazineName', 'The New Yorker'), ('workerName', 'Lewis Mumford'), ('books', 'Herman Melville; Renewal of Life')]
[('magazineName', 'The New Yorker'), ('workerName', 'Chris Ware'), ('books', 'Acme Novelty Library; Building Stories; Jimmy Corrigan, the Smartest Kid on Earth')]
[('magazineName', 'The New Yorker'), ('workerName', 'Hannah Arendt'), ('books', 'Crises of the Republic; The Three Escapes of Hannah Arendt')]


3

### Final results part 4
As in the first part 4 result the notable workers (in this case they have been considerated  belongs to "The New Yorker" but in this case they are only 3 and not 15.

## Part 5
### Is any employee of The New Yorker ever been nominated for a prize or award?

### Find workers who won a prize or award
wdt:P166 = award received

In [46]:
queryString = """
SELECT DISTINCT ?personName ?worker
WHERE { 
    
    ?worker ?works wd:Q217305 ;
            wdt:P31 wd:Q5 ;
            wdt:P166 ?price .
               
    OPTIONAL { ?worker sc:name ?personName . }
    
    
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('personName', "John O'Hara"), ('worker', 'http://www.wikidata.org/entity/Q548345')]
[('personName', 'Mavis Gallant'), ('worker', 'http://www.wikidata.org/entity/Q2550824')]
[('personName', 'Hannah Arendt'), ('worker', 'http://www.wikidata.org/entity/Q60025')]
[('personName', 'Susan Orlean'), ('worker', 'http://www.wikidata.org/entity/Q273168')]
[('personName', 'E. B. White'), ('worker', 'http://www.wikidata.org/entity/Q361617')]
[('personName', 'Saul Steinberg'), ('worker', 'http://www.wikidata.org/entity/Q432856')]
[('personName', 'Chris Ware'), ('worker', 'http://www.wikidata.org/entity/Q1077842')]
[('personName', 'Calvin Tomkins'), ('worker', 'http://www.wikidata.org/entity/Q2934539')]
[('personName', 'Jean-Jacques Sempé'), ('worker', 'http://www.wikidata.org/entity/Q354371')]
[('personName', 'Elizabeth Kolbert'), ('worker', 'http://www.wikidata.org/entity/Q5363086')]


10

### See the price names

In [47]:
queryString = """
SELECT DISTINCT ?personName (GROUP_CONCAT(?priceName; SEPARATOR="; ") as ?prices)
WHERE { 
    
    ?worker ?works wd:Q217305 ;
            wdt:P31 wd:Q5 ;
            wdt:P166 ?price .
    OPTIONAL { ?price sc:name ?priceName. }
               
    OPTIONAL { ?worker sc:name ?personName . }
    
    
} GROUP BY ?personName ?worker
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('personName', 'Elizabeth Kolbert'), ('prices', 'Guggenheim Fellowship; Heinz Award; Lannan Literary Awards; Science Journalism Award - Magazine; National Magazine Award')]
[('personName', 'Lawrence Wright'), ('prices', 'Lionel Gelber Prize; Lionel Gelber Prize; Pulitzer Prize for General Non-Fiction; Pulitzer Prize for General Non-Fiction; Los Angeles Times Book Prize; Los Angeles Times Book Prize; J. Anthony Lukas Book Prize; J. Anthony Lukas Book Prize; Helen Bernstein Book Award for Excellence in Journalism; Helen Bernstein Book Award for Excellence in Journalism')]
[('personName', 'Ryan Lizza'), ('prices', 'Aldo Beckman Memorial Award; Edwin M. Hood Award for Diplomatic Correspondence')]
[('personName', 'Bernard Lewis'), ('prices', 'George Polk Award')]
[('personName', 'Masha Gessen'), ('prices', 'Leipzig Book Award for European Understanding; National Book Award for Nonfiction; Kurt Tucholsky Prize')]
[('personName', 'Lizzie Presser'), ('prices', 'George Polk Award')]
[(

10

### Search nominated workers

In [48]:
queryString = """
SELECT DISTINCT ?personName ?pName ?p
WHERE { 
    
    ?worker ?p ?o .
    
    ?p sc:name ?pName .
    
    FILTER regex(?pName, ".*nomin.*")
        
    {
        SELECT DISTINCT ?worker WHERE{
        ?worker ?works wd:Q217305 ;
                wdt:P31 wd:Q5 .
        }           
    }
    
    OPTIONAL { ?worker sc:name ?personName . }  
} 
"""

print("Results")
run_query(queryString)

Results
[('personName', 'James Thurber'), ('pName', 'nominated for'), ('p', 'http://www.wikidata.org/prop/direct/P1411')]
[('personName', 'Mavis Gallant'), ('pName', 'nominated for'), ('p', 'http://www.wikidata.org/prop/direct/P1411')]
[('personName', 'Hannah Arendt'), ('pName', 'nominated for'), ('p', 'http://www.wikidata.org/prop/direct/P1411')]
[('personName', 'Chris Ware'), ('pName', 'nominated for'), ('p', 'http://www.wikidata.org/prop/direct/P1411')]
[('personName', 'Atul Gawande'), ('pName', 'nominated for'), ('p', 'http://www.wikidata.org/prop/direct/P1411')]
[('personName', 'Edmund Wilson'), ('pName', 'nominated for'), ('p', 'http://www.wikidata.org/prop/direct/P1411')]
[('personName', 'Alan Campbell'), ('pName', 'Academy Awards Database nominee ID'), ('p', 'http://www.wikidata.org/prop/direct/P6150')]
[('personName', 'Alan Campbell'), ('pName', 'nominated for'), ('p', 'http://www.wikidata.org/prop/direct/P1411')]
[('personName', 'Dorothy Parker'), ('pName', 'nominated for'), 

16

### Price name and nominated wdt:P1411

In [49]:
queryString = """
SELECT DISTINCT ?personName (GROUP_CONCAT(?priceName; separator="; ") AS ?nominatedFor)
WHERE { 
             
    ?worker ?works wd:Q217305 ;
            wdt:P31 wd:Q5 ;
            wdt:P1411 ?price .
    
    OPTIONAL { ?price sc:name ?priceName .}
    OPTIONAL { ?worker sc:name ?personName . }  
} 
"""

print("Results")
run_query(queryString)

Results
[('personName', 'Mavis Gallant'), ('nominatedFor', "Governor General's Award for English-language fiction")]
[('personName', 'David Remnick'), ('nominatedFor', 'Pulitzer Prize for General Non-Fiction; Pulitzer Prize for General Non-Fiction; Pulitzer Prize for International Reporting; Pulitzer Prize for International Reporting')]
[('personName', 'James Thurber'), ('nominatedFor', 'Nobel Prize in Literature')]
[('personName', 'Edmund Wilson'), ('nominatedFor', 'Nobel Prize in Literature; National Book Award for Nonfiction')]
[('personName', 'Dorothy Parker'), ('nominatedFor', 'Academy Award for Best Story; Academy Award for Best Writing, Adapted Screenplay')]
[('personName', 'Alan Campbell'), ('nominatedFor', 'Academy Award for Best Writing, Adapted Screenplay')]
[('personName', 'Jill Lepore'), ('nominatedFor', 'Pulitzer Prize for History; Pulitzer Prize for History; National Book Award for Nonfiction; National Book Award for Nonfiction')]
[('personName', 'Peter Hessler'), ('nomi

14

In [50]:
queryString = """
ASK WHERE { 
             
    ?worker ?works wd:Q217305 ;
            wdt:P31 wd:Q5 ;
            wdt:P1411 ?price .
} 
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': True}

### Results
As we can see from the last query that there is at least one "The New Yorker" worker has been nominated for a price or award.

In the previous query you can also see their name and the price or award where they has been nominated.